# CT head

CT study of a cadaver head from: <https://graphics.stanford.edu/data/voldata/>

In [1]:
import glob
import tarfile
import tempfile

import libcarna
import numpy as np
import pooch
import tifffile

Get the data:

In [2]:
data_tar_gz_filepath = pooch.retrieve(
    'https://graphics.stanford.edu/'
    'data/voldata/cthead-16bit.tar.gz',
    known_hash='md5:ea5874800bc1321fecd65ee791ca157b',
)
with tempfile.TemporaryDirectory() as data_dir_path:
    with tarfile.open(data_tar_gz_filepath) as data_tar_gz:
        data_tar_gz.extractall(
            data_dir_path,
            filter=tarfile.fully_trusted_filter,
        )
    data = np.dstack(
        [
            tifffile.imread(filepath) for filepath in
            sorted(glob.glob(f'{data_dir_path}/cthead-*.tif'))
        ]
    )
data.shape, data.dtype

((256, 256, 99), dtype('uint16'))

Visualize the data:

In [3]:
GEOMETRY_TYPE_VOLUME = 2

# Create and configure frame renderer
mip = libcarna.mip(GEOMETRY_TYPE_VOLUME, sr=400)
r = libcarna.renderer(600, 450, [mip])

# Create and configure scene
root = libcarna.node()
libcarna.volume(
    GEOMETRY_TYPE_VOLUME,
    data / data.max(),
    parent=root,
    spacing=(1, 1, 2),
    local_transform=libcarna.rotate('z', 90).rotate('x', 90),
)
camera = libcarna.camera(
    parent=root,
    projection=r.frustum(fov=90, z_near=10, z_far=1000),
    local_transform=libcarna.translate(0, 0, 300),
)

# Render
libcarna.imshow(r.render(camera))

In [4]:
# Render as animation
libcarna.imshow(
    libcarna.animation(
        [
            libcarna.animation.rotate_local(camera)
        ],
        n_frames=50,
    ).render(r, camera),
)